In [1]:
import torch
from torch import nn
import pandas as pd
import sklearn
import numpy as np

In [2]:

s_nn = nn.Sequential(
        nn.Linear(18,128),
        nn.LeakyReLU(),
        nn.Linear(128,128),
        nn.LeakyReLU(),
        nn.Linear(128,64),
        nn.LeakyReLU(),
        nn.Linear(64,32),
        nn.LeakyReLU(),
        nn.Linear(32,16),
        nn.LeakyReLU(),
        nn.Linear(16,8),
        nn.Linear(8,2),
        nn.LeakyReLU(),

    )

In [3]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(y_pred)

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc.item()

In [4]:
#data = pd.read_csv('/content/ML_Project/datasets/EDA.csv')
data = pd.read_csv('datasets/EDA.csv')
data = data.iloc[: , 1:]

In [5]:
df = data

In [6]:
df = df.drop(columns = ["SalesAmountInEuro","product_price","time_delay_for_conversion","click_timestamp","day","day_time","user_id","partner_id","product_id","product_title"])

In [7]:
df.head(5)

,Sale,nb_clicks_1week,product_age_group,device_type,audience_id,product_gender,product_brand,product_category(1),product_category(2),product_category(3),product_category(4),product_category(5),product_category(6),product_country,day_time_category,tree_encode,category_encode,nb_clicks_1week_category,product_price_category
0,0.0,0.0,0,3,0,0,0,22,145,699,910,442,89,3,5,6,63,0,0
1,0.0,0.0,0,3,0,0,0,22,145,699,910,442,89,3,2,6,63,0,0
2,0.0,0.0,1,1,0,1,1,1,1,699,910,442,89,1,17,12,63,0,0
3,0.0,0.0,0,2,0,0,0,22,145,699,910,442,89,3,21,6,63,0,0
4,0.0,2.0,1,3,0,2,2,2,2,1,1,442,89,2,21,18,63,2,0


In [8]:
from sklearn import preprocessing
y = df['Sale'].to_numpy()
X = df.drop(['Sale'], axis=1).to_numpy()
# min_max_scaler = preprocessing.MinMaxScaler()
# X = min_max_scaler.fit_transform(X)
y.shape

(100000,)

In [9]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test= train_test_split(X,
                                                y,
                                                test_size=0.25,
                                                random_state=42,
                                                shuffle=True)


In [10]:
print(x_train.shape)

(75000, 18)


In [11]:
from collections import Counter
count=Counter(y_train)
print(count)
class_count=np.array([count[0],count[1]])
print(class_count)
weight=1./class_count
print(weight)
samples_weight = np.array([weight[int(t)] for t in y_train])
samples_weight = torch.from_numpy(samples_weight)

Counter({0.0: 64699, 1.0: 10301})
[64699 10301]
[1.54561894e-05 9.70779536e-05]


In [12]:
sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weight, len(samples_weight))

In [13]:
from torch.utils.data import DataLoader, TensorDataset

train_subset = torch.utils.data.TensorDataset((torch.unsqueeze(torch.Tensor(x_train),1)), torch.Tensor(y_train))
val_subset = torch.utils.data.TensorDataset((torch.unsqueeze(torch.Tensor(x_test),1)), torch.LongTensor(y_test))

In [14]:

# dataset = TensorDataset(torch.unsqueeze(torch.Tensor(X),1),torch.Tensor(y))
# train_subset, val_subset = torch.utils.data.random_split(
#         dataset, [60000, 40000])

In [15]:


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
s_nn.to(device)
print(torch.cuda.is_available())

loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(s_nn.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)



BATCH_SIZE = 128
NUMBER_OF_EPOCHS = 20
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, sampler = sampler)
val_loader = DataLoader(dataset=val_subset, shuffle=False, batch_size=BATCH_SIZE)

False


In [16]:
from statistics import mean
all_train_losses = []
all_train_accuracy = []
all_val_losses = []
all_val_accuracy = []
for epoch in range(NUMBER_OF_EPOCHS):
  # training
  epoch_loss = 0
  acc_list = []
  s_nn.train()
  for i , (images , labels) in enumerate (train_loader):
    images , labels = images.to(device) , labels.to(device,dtype = torch.long)
    optimizer.zero_grad()
    outputs = s_nn(images)
    outputs = outputs.squeeze(1)
    loss = loss_function (outputs , labels)
    loss.backward()
    optimizer.step()
    # calculate accuracy
    epoch_loss += outputs.shape[0] * loss.item()
    total = labels.size(0)
    predicted = torch.max(outputs.data,1)[1]
    correct = (predicted == labels).sum().item()
    acc_list.append(correct/total)
  all_train_losses.append(epoch_loss/60000)
  print(epoch_loss/60000)
  all_train_accuracy.append(mean(acc_list))
  scheduler.step()
  # evaluation on validation data
  loss_list = []
  s_nn.eval() 
  with torch.no_grad():
    corr = 0
    tot = 0
    for j,(images , labels) in enumerate (val_loader):
      images , labels = images.to(device) , labels.to(device,dtype = torch.long)
      out = s_nn(images)
      out = out.squeeze(1).squeeze(1)
      # labels = labels.unsqueeze(1)
      loss_list.append(loss_function (out , labels).item())
      tot += labels.size(0)
      predicts = torch.max(out.data,1)[1]
      corr += (predicts == labels).sum().item()
    all_val_accuracy.append(corr/tot)
    all_val_losses.append(mean(loss_list))
    print("validation accuracy : "+str((corr/tot)*100))

0.8652366914510727
validation accuracy : 13.468
0.8619589856942494
validation accuracy : 59.86
0.8572608842611313
validation accuracy : 63.959999999999994
0.8537217051664988
validation accuracy : 63.512
0.8497953368743261
validation accuracy : 33.408
0.8484210662603379
validation accuracy : 58.748
0.8470488877058029
validation accuracy : 70.952
0.8462930446863175
validation accuracy : 64.89200000000001
0.8429094798008601
validation accuracy : 67.676
0.8406065933624903
validation accuracy : 68.22399999999999
0.8405349234104157
validation accuracy : 55.584
0.841249366402626
validation accuracy : 69.092
0.838599046754837
validation accuracy : 35.276
0.8386789299567541
validation accuracy : 65.92
0.8392666756073633
validation accuracy : 61.06399999999999
0.8485668324947357
validation accuracy : 70.648
0.8419020616054534
validation accuracy : 70.04
0.8375922733306885
validation accuracy : 71.336
0.8366739007472992
validation accuracy : 63.856
0.8365195874611536
validation accuracy : 66.044


In [21]:
s_nn.eval()
xt = torch.Tensor(x_test)
xt = xt.to(device)

t = s_nn(xt)

In [18]:
t.shape

torch.Size([25000, 2])

In [19]:
predicted = torch.max(t.data,1)[1]

In [23]:
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score, accuracy_score
confusion_matrix(predicted.cpu(), y_test)
print(f1_score(predicted.cpu(), y_test))
print(precision_score(predicted.cpu(), y_test))
print(accuracy_score(predicted.cpu(), y_test))



0.26393826411167953
0.4529761904761905
0.66044
